In [1]:
import os
import sys
import pathlib
import pandas as pd
import numpy as np
from app_tool.utils.utils_models import preprocess_data, predict_adicion
from pycaret.classification import load_model
from utils.bq_conn import bq_conn_file

sys.path.append(
    os.path.join(pathlib.Path(os.getcwd()))
)

## Test Monto Estimado

In [7]:
features = {'tipo_de_contrato': "Compraventa", 
            'departamento_ejecucion':"Antioquia",
            'id_grupo': "E", 
            'dias_ejec_final_contrato': 10,
            'cuantia_proceso': 204000,
            'id_objeto_a_contratar': 49000000
            }

In [8]:
data_monto = preprocess_data(features=features, type="monto")
data_adicion = preprocess_data(features=features, type="adicion")

In [9]:
# Load model
model = load_model('app_tools/models/lightgbm_model_monto_recomendado')
model2 = load_model('app_tools/models/RFClassifier_adicion')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [11]:
predict_adicion(model2, data_adicion)

0.7

## Test data Secop

In [2]:
pd.options.display.max_columns = None

bq_client = bq_conn_file(project_id="dsa-miad-365616")

query = '''
SELECT
    *
FROM `Proyecto.DatosSecopI`
ORDER BY anno_cargue_secop DESC
'''
bq_client = bq_conn_file(project_id="dsa-miad-365616")
secop_I = bq_client.execute_query(query=query)
print(f"El número de filas del dataset: {secop_I.shape[0]}\nEl número de columnas del dataset: {secop_I.shape[1]}")
secop_I.head(5)

El número de filas del dataset: 26553
El número de columnas del dataset: 38


,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,dias_ejec_final_contrato,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,191389136418143424,2019,13,contratacion minima cuantia,celebrado,50000000,contratar servicio suministro elementos materi...,suministro,facatativa,facatativa,cundinamarca,CPMMSFFAMCNo01DE2019,CPMMSFFAMCNO01DE2019,50000000,E,productos de uso final,5022,productos de cereales legumbres,502213,harina productos de molinos,8143424,nit de persona juridica,900791326,inversiones ocazo,cedula de ciudadania,1022363447,330,330,D,0,0,50000000,0,0,50000000,peso colombiano,contratar compra elementos materia prima para ...,{'url': 'https://www.contratos.gov.co/consulta...
1,1912101041779226352,2019,12,contratacion directa ley de,celebrado,53000000,adquisicion uniformes para dotar personal cuer...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA184DE2019,184DE2019,22980096902,E,productos de uso final,5310,ropa,531027,uniformes,9226352,nit de persona juridica,860020227,fondo rotatorio de la policia forpo,nit de persona juridica,860020227,660,690,D,1,30,22980096902,0,0,22980096902,peso colombiano,adquisicion uniformes para dotar personal cuer...,{'url': 'https://www.contratos.gov.co/consulta...
2,1912101443259265538,2019,12,contratacion directa ley de,celebrado,55000000,contratar diseno produccion tarjetas presentac...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA188DE2019,188DE2019,35998452,E,productos de uso final,5510,medios impresos,551015,publicaciones impresas,9265538,nit de persona juridica,830001113,imprenta nacional de colombia,nit de persona juridica,830001113,31,71,D,1,40,35998452,0,0,35998452,peso colombiano,contratar diseno produccion tarjetas presentac...,{'url': 'https://www.contratos.gov.co/consulta...
3,191294985788644918,2019,12,contratacion directa ley de,liquidado,72000000,prestar servicio mantenimiento preventivo corr...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA106DE2019,106DE2019,16767536,F,servicios,7210,servicios de mantenimiento reparaciones de con...,721015,servicios de apoyo para la construccion,8644918,nit de persona juridica,860025639,mitsubishi electric de colombia limitada,nit de persona juridica,860025639,170,170,D,0,0,16767536,0,0,16767536,peso colombiano,prestar servicio mantenimiento preventivo corr...,{'url': 'https://www.contratos.gov.co/consulta...
4,191988590538059861,2019,19,asociacion publico privada,celebrado,73000000,desintegracion vehiculos automotores estado in...,no encontrado en hoja proceso,bogota,bogota,bogota,CONVENIODEASOCIACIÓN231DE2018,231DE2018,0,F,servicios,7318,servicios de maquinado procesado,731810,servicios de maquinado,8059861,nit de persona juridica,806011019,recuperaciones naranjo recycling sas,cedula de ciudadania,57439563,360,360,D,0,0,0,0,0,0,peso colombiano,desintegracion vehiculos automotores estado in...,{'url': 'https://www.contratos.gov.co/consulta...


In [3]:
input_models = secop_I.filter(items=['tipo_de_contrato', 
                'id_grupo', 
                'cuantia_proceso',
                'departamento_ejecucion',
                'plazo_de_ejec_del_contrato_',
                'id_objeto_a_contratar',
                                      ])
input_models.head(5)

,tipo_de_contrato,id_grupo,cuantia_proceso,departamento_ejecucion,plazo_de_ejec_del_contrato_,id_objeto_a_contratar
0,suministro,E,50000000,cundinamarca,330,50000000
1,prestacion de servicios,E,22980096902,bogota,660,53000000
2,prestacion de servicios,E,35998452,bogota,31,55000000
3,prestacion de servicios,F,16767536,bogota,170,72000000
4,no encontrado en hoja proceso,F,0,bogota,360,73000000


In [6]:
input_models.plazo_de_ejec_del_contrato_.max()

7500

In [7]:
input_models.plazo_de_ejec_del_contrato_.min()

0

In [13]:
data= pd.read_parquet('app_tool/info_data/objeto_a_contratar.parquet')
print(data["Id Objeto a Contratar"].to_list())

['10000000', '11000000', '12000000', '13000000', '14000000', '15000000', '20000000', '21000000', '22000000', '23000000', '24000000', '25000000', '26000000', '27000000', '30000000', '31000000', '32000000', '39000000', '40000000', '41000000', '42000000', '43000000', '44000000', '45000000', '46000000', '47000000', '48000000', '49000000', '50000000', '51000000', '52000000', '53000000', '54000000', '55000000', '56000000', '60000000', '70000000', '71000000', '72000000', '73000000', '76000000', '77000000', '78000000', '80000000', '81000000', '82000000', '83000000', '84000000', '85000000', '86000000', '90000000', '91000000', '92000000', '93000000', '94000000', '95000000']


In [9]:
from pickle import load
path = os.path.join(os.getcwd(),'app_tools', 'utils')
preprocessor = load(open(f'{path}/preprocessor_prorroga.pkl', 'rb'))
data_scaler = preprocessor.transform(input_models)
data_scaler.head()

,cuantia_proceso,plazo_de_ejec_del_contrato_,tipo_de_contrato_acuerdo marco,tipo_de_contrato_arrendamiento,tipo_de_contrato_comodato,tipo_de_contrato_compraventa,tipo_de_contrato_concesion,tipo_de_contrato_consultoria,tipo_de_contrato_no encontrado en hoja proceso,tipo_de_contrato_obra,tipo_de_contrato_otro tipo de contrato,tipo_de_contrato_prestacion de servicios,tipo_de_contrato_suministro,id_grupo_A,id_grupo_B,id_grupo_C,id_grupo_D,id_grupo_E,id_grupo_F,id_grupo_G,departamento_ejecucion_amazonas,departamento_ejecucion_antioquia,departamento_ejecucion_arauca,departamento_ejecucion_atlantico,departamento_ejecucion_bogota,departamento_ejecucion_bolivar,departamento_ejecucion_boyaca,departamento_ejecucion_caldas,departamento_ejecucion_caqueta,departamento_ejecucion_casanare,departamento_ejecucion_cauca,departamento_ejecucion_cesar,departamento_ejecucion_choco,departamento_ejecucion_colombia,departamento_ejecucion_cordoba,departamento_ejecucion_cundinamarca,departamento_ejecucion_huila,departamento_ejecucion_la guajira,departamento_ejecucion_magdalena,departamento_ejecucion_meta,departamento_ejecucion_narino,departamento_ejecucion_norte de santander,departamento_ejecucion_otros paises,departamento_ejecucion_putumayo,departamento_ejecucion_quindio,departamento_ejecucion_risaralda,departamento_ejecucion_san andres,departamento_ejecucion_santander,departamento_ejecucion_sucre,departamento_ejecucion_tolima,departamento_ejecucion_valle del cauca,id_objeto_a_contratar_10000000,id_objeto_a_contratar_11000000,id_objeto_a_contratar_12000000,id_objeto_a_contratar_13000000,id_objeto_a_contratar_14000000,id_objeto_a_contratar_15000000,id_objeto_a_contratar_20000000,id_objeto_a_contratar_21000000,id_objeto_a_contratar_22000000,id_objeto_a_contratar_23000000,id_objeto_a_contratar_24000000,id_objeto_a_contratar_25000000,id_objeto_a_contratar_26000000,id_objeto_a_contratar_27000000,id_objeto_a_contratar_30000000,id_objeto_a_contratar_31000000,id_objeto_a_contratar_32000000,id_objeto_a_contratar_39000000,id_objeto_a_contratar_40000000,id_objeto_a_contratar_41000000,id_objeto_a_contratar_42000000,id_objeto_a_contratar_43000000,id_objeto_a_contratar_44000000,id_objeto_a_contratar_45000000,id_objeto_a_contratar_46000000,id_objeto_a_contratar_47000000,id_objeto_a_contratar_48000000,id_objeto_a_contratar_49000000,id_objeto_a_contratar_50000000,id_objeto_a_contratar_51000000,id_objeto_a_contratar_52000000,id_objeto_a_contratar_53000000,id_objeto_a_contratar_54000000,id_objeto_a_contratar_55000000,id_objeto_a_contratar_56000000,id_objeto_a_contratar_60000000,id_objeto_a_contratar_70000000,id_objeto_a_contratar_71000000,id_objeto_a_contratar_72000000,id_objeto_a_contratar_73000000,id_objeto_a_contratar_76000000,id_objeto_a_contratar_77000000,id_objeto_a_contratar_78000000,id_objeto_a_contratar_80000000,id_objeto_a_contratar_81000000,id_objeto_a_contratar_82000000,id_objeto_a_contratar_83000000,id_objeto_a_contratar_84000000,id_objeto_a_contratar_85000000,id_objeto_a_contratar_86000000,id_objeto_a_contratar_90000000,id_objeto_a_contratar_91000000,id_objeto_a_contratar_92000000,id_objeto_a_contratar_93000000,id_objeto_a_contratar_94000000,id_objeto_a_contratar_95000000
0,0.183652,1.831428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,138.304964,4.239830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0

In [7]:
# Load model
model_prorroga = load_model('app_tools/models/xgboost_prorroga')

Transformation Pipeline and Model Successfully Loaded


In [10]:
from pycaret.classification import predict_model
predictions_data = predict_model(estimator = model_prorroga, data = data_scaler)

In [12]:
predictions_data.head()

,cuantia_proceso,plazo_de_ejec_del_contrato_,tipo_de_contrato_acuerdo marco,tipo_de_contrato_arrendamiento,tipo_de_contrato_comodato,tipo_de_contrato_compraventa,tipo_de_contrato_concesion,tipo_de_contrato_consultoria,tipo_de_contrato_no encontrado en hoja proceso,tipo_de_contrato_obra,tipo_de_contrato_otro tipo de contrato,tipo_de_contrato_prestacion de servicios,tipo_de_contrato_suministro,id_grupo_A,id_grupo_B,id_grupo_C,id_grupo_D,id_grupo_E,id_grupo_F,id_grupo_G,departamento_ejecucion_amazonas,departamento_ejecucion_antioquia,departamento_ejecucion_arauca,departamento_ejecucion_atlantico,departamento_ejecucion_bogota,departamento_ejecucion_bolivar,departamento_ejecucion_boyaca,departamento_ejecucion_caldas,departamento_ejecucion_caqueta,departamento_ejecucion_casanare,departamento_ejecucion_cauca,departamento_ejecucion_cesar,departamento_ejecucion_choco,departamento_ejecucion_colombia,departamento_ejecucion_cordoba,departamento_ejecucion_cundinamarca,departamento_ejecucion_huila,departamento_ejecucion_la guajira,departamento_ejecucion_magdalena,departamento_ejecucion_meta,departamento_ejecucion_narino,departamento_ejecucion_norte de santander,departamento_ejecucion_otros paises,departamento_ejecucion_putumayo,departamento_ejecucion_quindio,departamento_ejecucion_risaralda,departamento_ejecucion_san andres,departamento_ejecucion_santander,departamento_ejecucion_sucre,departamento_ejecucion_tolima,departamento_ejecucion_valle del cauca,id_objeto_a_contratar_10000000,id_objeto_a_contratar_11000000,id_objeto_a_contratar_12000000,id_objeto_a_contratar_13000000,id_objeto_a_contratar_14000000,id_objeto_a_contratar_15000000,id_objeto_a_contratar_20000000,id_objeto_a_contratar_21000000,id_objeto_a_contratar_22000000,id_objeto_a_contratar_23000000,id_objeto_a_contratar_24000000,id_objeto_a_contratar_25000000,id_objeto_a_contratar_26000000,id_objeto_a_contratar_27000000,id_objeto_a_contratar_30000000,id_objeto_a_contratar_31000000,id_objeto_a_contratar_32000000,id_objeto_a_contratar_39000000,id_objeto_a_contratar_40000000,id_objeto_a_contratar_41000000,id_objeto_a_contratar_42000000,id_objeto_a_contratar_43000000,id_objeto_a_contratar_44000000,id_objeto_a_contratar_45000000,id_objeto_a_contratar_46000000,id_objeto_a_contratar_47000000,id_objeto_a_contratar_48000000,id_objeto_a_contratar_49000000,id_objeto_a_contratar_50000000,id_objeto_a_contratar_51000000,id_objeto_a_contratar_52000000,id_objeto_a_contratar_53000000,id_objeto_a_contratar_54000000,id_objeto_a_contratar_55000000,id_objeto_a_contratar_56000000,id_objeto_a_contratar_60000000,id_objeto_a_contratar_70000000,id_objeto_a_contratar_71000000,id_objeto_a_contratar_72000000,id_objeto_a_contratar_73000000,id_objeto_a_contratar_76000000,id_objeto_a_contratar_77000000,id_objeto_a_contratar_78000000,id_objeto_a_contratar_80000000,id_objeto_a_contratar_81000000,id_objeto_a_contratar_82000000,id_objeto_a_contratar_83000000,id_objeto_a_contratar_84000000,id_objeto_a_contratar_85000000,id_objeto_a_contratar_86000000,id_objeto_a_contratar_90000000,id_objeto_a_contratar_91000000,id_objeto_a_contratar_92000000,id_objeto_a_contratar_93000000,id_objeto_a_contratar_94000000,id_objeto_a_contratar_95000000,prediction_label,prediction_score
0,0.183652,1.831428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.9966
1,138.304962,4.239830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,